In [2]:
# auto reload libraries (you do need to re-import libraries if you make changes)
%load_ext autoreload
%autoreload 2

# base 
import pandas as pd
import numpy as np
from pprint import pprint

# preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer,KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import TruncatedSVD


# models
from sklearn.ensemble import RandomForestRegressor

# own defined functions/classes 
from preprocessing.preprocess_pipe import MakeLowerCase
from preprocessing.preprocess_pipe import HdResolutionCategorizer
from preprocessing.preprocess_pipe import StorageCategorizer
from preprocessing.preprocess_pipe import SsdCategorizer

from preprocessing.preprocess_pipe import print_missing
from preprocessing.preprocess_pipe import calculate_perf
from preprocessing.preprocess_pipe import custom_scoring_func


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# global parameters
pd.set_option('display.max_columns', 500)

## TODO
- Add custum scoring metric: **DONE**
- further integrate preprocessing steps from Arnaud **DONE**
- Integerate steps from Victor **Partly DONE**
- Integrate steps from Bram 
- Integrate custum scoring metric (also when performing the cross validation/grid search)
-  Make function that checkts whether prediction of the maximum price are >= predictions of the minimum price
- Implement better missing values imputations methods
- Add some post processing visualizations:
    - feature importance plots
    - look at our predictions visually (do the make since?)
    - look at the residuals
    - Have a look at this especially section 5 for model interpretability ideas
    https://christophm.github.io/interpretable-ml-book/pdp.html

# Step 1: import data and split in train and validation set
The validation set is more our own kind of test set

In [4]:
# read in trainig and validation data
# use the same data split as we did in R
df_all_train = pd.read_csv("../data/train.csv", sep=',')
df_test = pd.read_csv("../data/test.csv", sep=',')

print(f'Dimensions of all training data {df_all_train.shape}')
print(f'Dimension test data {df_test.shape}')

Dimensions of all training data (510, 22)
Dimension test data (222, 20)


In [5]:
# split in training and validation set

df_train = df_all_train.sample(frac=0.75, random_state=0)
df_val = df_all_train.drop(df_train.index)

# reset index, if you don't resit missing rows get inserted in the pipeline
# see: https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
df_train = df_train.reset_index()
df_val = df_val.reset_index()

# Step 2: define pipeline

The pipeline should handle all steps performed on the data from data cleaning till the acutall models. 
It should also be easily fairly easily to apply this to the test data. Therefore we make use of the scikit learn pipeline which allows all of this functionality. You also implement your own `custom` transformations.

Make a difference between the pre processing steps for
- numerical features
- categorical features

I still don't use all features, since some extra data cleaning is needed on certain features

In [6]:
#Numerical features to pass down the numerical pipeline 
numerical_features = ['screen_size','detachable_keyboard' , 
                      'ram', 'weight','pixels_x','pixels_y']

#Categrical features to pass down the categorical pipeline 
categorical_features = ['brand', 'screen_surface','touchscreen',
                        'cpu', 'discrete_gpu','gpu', 'os','ssd','storage']

# define all unique features
features = np.unique(numerical_features).tolist() + np.unique(categorical_features).tolist()

# target variables
target = ['min_price','max_price']

In [7]:
# train
X_train = df_train[features]
y_train = df_train[target]

# validation (this is kind our own test set)
X_val  = df_val[features]
y_val = df_val[target]

# train_validation (this is all training data we have) for fitting the model
X_all_train = df_all_train[features]
y_all_train = df_all_train[target]

# test
X_test = df_test[features]

In [9]:
# you can add many more and 
# you can even define custom preprocessing steps like 'MakeLowerCase()'

# pipeline  uses only numerical features,
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    #('imputer', KNNImputer(n_neighbors=5, weights='uniform',metric='nan_euclidean')),
    ('imputer', IterativeImputer(missing_values=np.nan, random_state=0, n_nearest_features=None, sample_posterior=True)),
    #('transformation', PowerTransformer(method='yeo-johnson',standardize=False)),
     #Scale features using statistics that are robust to outliers.
    ('scaler', RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0)))]) 

# pipeline use only categorical features
categorical_transformer = Pipeline(steps=[ 
    ('lowercase', MakeLowerCase()), # lower cases all columns containing strings
    ('sd_category' ,SsdCategorizer(drop_original_feature=True)),
    ('storage_category', StorageCategorizer()),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# add both preprocessing pipelines in one pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [10]:
# this is how the preporcessed pipeline looks like (just to have an idea)
pd.DataFrame(preprocessor.fit_transform(X_train)).shape

(382, 1)

# Step 3: add models to pipeline
## 1) Random Forest

### A) Training and parameter tuning

In [11]:
# define model: I just add some default parameters but you could
# also just write: RandomForestRegressor() since we will perform a grid search 
# to find good hyperparameter values
model_rf = RandomForestRegressor(random_state=1)

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model_rf.get_params())

# add to pipeline
pipeline_rf = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_rf)])


# add transformation on the target variable, by default power transformation 
# also performs standardization after performing the power transformation
# and back transform to the original space when outputting predictions 
transformer_target = PowerTransformer(method='yeo-johnson',standardize=True)
pipeline_rf_update = TransformedTargetRegressor(regressor=pipeline_rf, 
                                         transformer=transformer_target)

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}


In [13]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 6000, num = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 200, num = 50)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 5, 10,20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid_rf = {
               'regressor__regressor__n_estimators': n_estimators,
               'regressor__regressor__max_features': max_features,
               'regressor__regressor__max_depth': max_depth,
               'regressor__regressor__min_samples_split': min_samples_split,
               'regressor__regressor__min_samples_leaf': min_samples_leaf,
               'regressor__regressor__bootstrap': bootstrap}

#### Helpful articles:
 - custom scoring metric: https://stackoverflow.com/questions/48468115/how-to-create-a-customized-scoring-function-in-scikit-learn-for-scoring-a-set-of
 - random parameter search: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
you can also use the traditional grid search in pyhton, but I prefer the randomizedSearch
 - Once the optimal parameters are found for the model, you don't need to run this again

In [14]:
# define random search (and narrow down time grid search)
rf_random_search = RandomizedSearchCV(
   estimator = pipeline_rf_update, 
   param_distributions = random_grid_rf, n_iter = 100,  
   cv = 10, verbose=2, random_state=1, n_jobs = -1, refit=True,
   scoring=make_scorer(custom_scoring_func, greater_is_better=False)
)


# run grid search and refit with best hyper parameters
rf_random_search.fit(X_train, y_train)  
print(rf_random_search.best_params_)    
print(rf_random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 26.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 44.5min finished


{'regressor__regressor__n_estimators': 2857, 'regressor__regressor__min_samples_split': 5, 'regressor__regressor__min_samples_leaf': 1, 'regressor__regressor__max_features': 'sqrt', 'regressor__regressor__max_depth': 181, 'regressor__regressor__bootstrap': False}
-315.5175356454738


- **TODO make a plot that visualizes hyperparameters (maybe kind of heatmap)**
- Once we have found good tuning parameters write them down so we don't need to redo this step over and over

In [15]:
# have look at the best hyperparameters and their respective performance (maybe also look at the sd)
pd.DataFrame(rf_random_search.cv_results_).sort_values(
    by=['mean_test_score'],ascending=False).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__regressor__n_estimators,param_regressor__regressor__min_samples_split,param_regressor__regressor__min_samples_leaf,param_regressor__regressor__max_features,param_regressor__regressor__max_depth,param_regressor__regressor__bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
93,7.756778,0.147370,0.381489,0.016976,2857,5,1,sqrt,181,False,"{'regressor__regressor__n_estimators': 2857, '...",-272.750763,-434.221200,-278.790106,-308.786471,-280.774726,-385.856308,-311.530852,-349.243861,-293.190614,-240.030456,-315.517536,55.514433,1
47,16.502742,0.254536,0.845860,0.132707,5663,5,1,sqrt,20,False,"{'regressor__regressor__n_estimators': 5663, '...",-272.978500,-433.775705,-278.319974,-308.040762,-281.107885,-385.856782,-312.665655,-347.969186,-293.861860,-240.764269,-315.534058,55.212333,2
29,12.073451,0.278458,0.399036,0.026090,2969,2,1,sqrt,192,False,"{'regressor__regressor__n_estimators': 2969, '...",-261.299327,-439.731069,-273.029449,-337.622471,-281.089347,-376.183880,-319.388937,-352.818326,-289.229114,-248.908424,-317.930035,56.572235,3
13,15.297682,1.709751,0.465761,0.128846,3081,2,1,sqrt,34,False,"{'regressor__regressor__n_estimators': 3081, '...",-261.388976,-439.496700,-272.828151,-337.578325,-280.806674,-376.390918,-319.592834,-353.011912,-288.974698,-249.270393,-317.933958,56.548617,4
38,11.764419,0.870570,0.510842,0.053234,3642,5,1,sqrt,67,True,"{'regressor__regressor__n_estimators': 3642, '...",-275.675132,-459.393618,-287.871329,-299.898764,-302.369528,-389.045133,-305.570318,-353.990018,-303.489289,-227.347639,-320.465077,61.747945,5


### B) Performance on validation data

In [16]:
calculate_perf(y_val, rf_random_search.predict(X_val))

{'minimum price': 118.96486417124208,
 'maximum price': 118.1037856365381,
 'total error': 237.0686498077802}

### C) Post processing

 - inspect predictions/residuals (make visualisations)
 - feature importance

### D) Predictions test data

Refit on all training data (using the parameters found on the random search) and submit prediction

In [18]:
# train your final model on all data with best parameters 
model_rf_final = RandomForestRegressor(
     n_estimators=2857, 
     max_depth=181,
     max_features='sqrt',
     min_samples_split=5,
     min_samples_leaf=1,
     bootstrap=False,
     n_jobs=-1
)
# add to pipeline
pipeline_rf_final = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_rf_final)],
              verbose=True)

# again add transformer for target variable
pipeline_rf_final = TransformedTargetRegressor(regressor=pipeline_rf_final, 
                                         transformer=transformer_target)

# fit final model on all training data we have at hand
pipeline_rf_final = pipeline_rf_final.fit(X_all_train, y_all_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   3.8s


In [19]:
# make predictions on test data
rf_pred_test = pipeline_rf_final.predict(X_test)

In [20]:
# submission format
rf_submission_format = pd.DataFrame.from_dict(
 {'ID':df_test['id'].values,
 'MIN':rf_pred_test[:,0],
 'MAX':rf_pred_test[:,1]}).set_index('ID')

In [21]:
# write to csv
rf_submission_format.to_csv('../output/predictions_test/rf_python.csv' ,
                            header=True, index=True, sep=',')